In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/30-years-of-european-solar-generation/EMHIRESPV_TSh_CF_Country_19862015.csv
/kaggle/input/30-years-of-european-solar-generation/emhirespv_gonzalezaparicioetal2017_newtemplate_corrected_last.pdf
/kaggle/input/30-years-of-european-solar-generation/EMHIRES_PVGIS_TSh_CF_n2_19862015.csv


In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import seaborn as sns
import numpy as np
import math
import os

In [28]:
df=pd.read_csv("/kaggle/input/30-years-of-european-solar-generation/EMHIRESPV_TSh_CF_Country_19862015.csv")

In [29]:
# Convert the string "01/01/1986 00:00" to a datetime object with the specified format
start_date = pd.to_datetime("01/01/1986 00:00", format="%m/%d/%Y %H:%M")

# Create an array of integers from 0 to the length of df4
hours = np.arange(len(df))

# Add the array of hours to the start date and assign the result to a new column 'Date' in df4
df['Date'] = start_date + pd.to_timedelta(hours, unit='h')

In [30]:
df = df.dropna() # Removes rows with null values

In [31]:
# Use the melt method to combine the state columns into a single "Countries" column
df = pd.melt(df, id_vars=['Date'], value_vars=['AT', 'BE','BG','CH','CY','CZ','DE','DK','EE','ES','FI','FR','EL','HR','HU','IE','IT','LT','LU','LV','NL','NO','PL','PT' ,'RO', 'SI', 'SK', 'SE', 'UK'], var_name='COUNTRY', value_name='ValueEmhires')

In [32]:
# Convert 'Date' to datetime and extract features
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Hour'] = df['Date'].dt.hour

# Remove the original 'Date' column
df = df.drop(['Date'], axis=1)

country_encoder = LabelEncoder()
df['COUNTRY'] = country_encoder.fit_transform(df['COUNTRY'])

# Apply one-hot encoding to the 'COUNTRY' column
df = pd.get_dummies(df, columns=['COUNTRY'], prefix='country')
df = pd.get_dummies(df, columns=['Year'], prefix='year')
df = pd.get_dummies(df, columns=['Month'], prefix='month')
df = pd.get_dummies(df, columns=['Day'], prefix='day')


# Convert boolean columns to numerical values (0 or 1).
boolean_columns = [
    'country_0', 'country_1', 'country_2', 'country_3', 'country_4', 'country_5',
    'country_6', 'country_7', 'country_8', 'country_9', 'country_10', 'country_11',
    'country_12', 'country_13', 'country_14', 'country_15', 'country_16', 'country_17',
    'country_18', 'country_19', 'country_20', 'country_21', 'country_22', 'country_23',
    'country_24', 'country_25', 'country_26', 'country_27', 'country_28',
    'year_1986', 'year_1987', 'year_1988', 'year_1989', 'year_1990', 'year_1991',
    'year_1992', 'year_1993', 'year_1994', 'year_1995', 'year_1996', 'year_1997',
    'year_1998', 'year_1999', 'year_2000', 'year_2001', 'year_2002', 'year_2003',
    'year_2004', 'year_2005', 'year_2006', 'year_2007', 'year_2008', 'year_2009',
    'year_2010', 'year_2011', 'year_2012', 'year_2013', 'year_2014', 'year_2015',
    'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
    'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7', 'day_8', 'day_9', 'day_10', 'day_11', 'day_12', 'day_13', 'day_14', 'day_15', 'day_16', 'day_17', 'day_18', 'day_19', 'day_20', 'day_21', 'day_22', 'day_23', 'day_24', 'day_25', 'day_26', 'day_27', 'day_28', 'day_29', 'day_30', 'day_31'
]


df[boolean_columns] = df[boolean_columns].astype(int)


# Print the resulting dataframe
print(df)

         ValueEmhires  Hour  country_0  country_1  country_2  country_3  \
0                 0.0     0          1          0          0          0   
1                 0.0     1          1          0          0          0   
2                 0.0     2          1          0          0          0   
3                 0.0     3          1          0          0          0   
4                 0.0     4          1          0          0          0   
...               ...   ...        ...        ...        ...        ...   
7626067           0.0    19          0          0          0          0   
7626068           0.0    20          0          0          0          0   
7626069           0.0    21          0          0          0          0   
7626070           0.0    22          0          0          0          0   
7626071           0.0    23          0          0          0          0   

         country_4  country_5  country_6  country_7  ...  day_22  day_23  \
0                0     

In [33]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7626072 entries, 0 to 7626071
Columns: 104 entries, ValueEmhires to day_31
dtypes: float64(1), int32(1), int64(102)
memory usage: 5.9 GB
None


## wandb link: https://wandb.ai/ales-2000-09/EMHIRES?workspace=user-ales-2000-09

In [34]:
import wandb

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Label")

wandb.login(key=secret_value_0)

wandb.init(project='EMHIRES', save_code=True)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ales-2000-09. Use `wandb login --relogin` to force relogin


In [35]:
# # Model definition
# class LinearRegressionModel(nn.Module):
#     def __init__(self, input_dim, output_dim):
#         super(LinearRegressionModel, self).__init__()
#         self.linear = nn.Linear(input_dim, output_dim)

#     def forward(self, x):
#         out = self.linear(x)
#         return out

In [36]:
# class RegressionModel(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim):
#         super(RegressionModel, self).__init__()
#         self.layer1 = nn.Linear(input_dim, hidden_dim)
#         self.relu = nn.ReLU()
#         self.layer2 = nn.Linear(hidden_dim, output_dim)

#     def forward(self, x):
#         out = self.layer1(x)
#         out = self.relu(out)
#         out = self.layer2(out)
#         return out


In [37]:
# class RegressionModel(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim):
#         super(RegressionModel, self).__init__()
#         self.fc1 = nn.Linear(input_dim, hidden_dim)
#         self.relu1 = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_dim, hidden_dim)
#         self.relu2 = nn.ReLU()
#         self.fc3 = nn.Linear(hidden_dim, output_dim)

#     def forward(self, x):
#         out = self.fc1(x)
#         out = self.relu1(out)
#         out = self.fc2(out)
#         out = self.relu2(out)
#         out = self.fc3(out)
#         return out


In [38]:
class RNNRegressionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNNRegressionModel, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.rnn(x)
        if len(out.shape) > 2:
            out = self.fc(out[:, -1, :])
        else:
            out = self.fc(out)
        return out.squeeze(1) if len(out.shape) > 2 else out

In [39]:
# Identify the 'ValueEmhires' columntarget_column = 'ValueEmhires'
target_index = df.columns.get_loc('ValueEmhires')

# Select all columns except 'ValueEmhires' as input
X = df.iloc[:, target_index+1:]

# Select only 'ValueEmhires' as the target
y = df['ValueEmhires']

print(X)
print(y)
# Split the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Number of input and output features
input_dim = X_train.shape[1]
output_dim = 1

# Initialize the model
#model = LinearRegressionModel(input_dim, output_dim)

# # Default Initialization
# nn.init.xavier_uniform_(model.linear.weight)
# nn.init.zeros_(model.linear.bias)

#model = RegressionModel(input_dim, input_dim*2, output_dim)
print(input_dim)
model = RNNRegressionModel(input_dim, input_dim*2, output_dim)


# Define the loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.00001)

print(df.info())

         Hour  country_0  country_1  country_2  country_3  country_4  \
0           0          1          0          0          0          0   
1           1          1          0          0          0          0   
2           2          1          0          0          0          0   
3           3          1          0          0          0          0   
4           4          1          0          0          0          0   
...       ...        ...        ...        ...        ...        ...   
7626067    19          0          0          0          0          0   
7626068    20          0          0          0          0          0   
7626069    21          0          0          0          0          0   
7626070    22          0          0          0          0          0   
7626071    23          0          0          0          0          0   

         country_5  country_6  country_7  country_8  ...  day_22  day_23  \
0                0          0          0          0  ...   

In [40]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7626072 entries, 0 to 7626071
Columns: 104 entries, ValueEmhires to day_31
dtypes: float64(1), int32(1), int64(102)
memory usage: 5.9 GB
None


In [41]:
# Training
epochs = 2
batch_size = 5000
wandb.log({'Epoch': epochs})
wandb.log({'batch_size': batch_size})

for epoch in range(epochs):
    model.train()
    for i in range(0, X_train.shape[0], batch_size):
        inputs = torch.from_numpy(X_train.iloc[i:i+batch_size].values).float()
        labels = torch.from_numpy(y_train.iloc[i:i+batch_size].values).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss_total = 0.0

    with torch.no_grad():
        for i in range(0, X_val.shape[0], batch_size):
            val_batch_inputs = torch.from_numpy(X_val.iloc[i:i+batch_size].values).float()
            val_batch_labels = torch.from_numpy(y_val.iloc[i:i+batch_size].values).float().unsqueeze(1)

            val_outputs = model(val_batch_inputs)
            val_loss = criterion(val_outputs, val_batch_labels)
            val_loss_total += val_loss.item()

    avg_val_loss = val_loss_total / (X_val.shape[0] / batch_size)

    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {loss.item()}, Validation Loss: {avg_val_loss}')
    wandb.log({'Training Loss': loss.item(), 'Validation Loss': avg_val_loss})


# Test
model.eval()
with torch.no_grad():
    test_inputs = torch.from_numpy(X_test.values).float()
    test_labels = torch.from_numpy(y_test.values).float().unsqueeze(1)
    test_outputs = model(test_inputs)
    test_loss = criterion(test_outputs, test_labels)

print(f'Test Loss: {test_loss.item()}')
wandb.log({'Test Loss': test_loss.item()})

wandb.finish()

Epoch 1/2, Training Loss: 0.04110394045710564, Validation Loss: 0.04230914172925139
Epoch 2/2, Training Loss: 0.03785869851708412, Validation Loss: 0.03948716109068664
Test Loss: 0.03929314762353897


wandb: WARNING No program path found when generating artifact job source for a non-colab notebook run. See https://docs.wandb.ai/guides/launch/create-job
wandb: WARNING Source type is set to 'artifact' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,▁
Test Loss,▁
Training Loss,█▁
Validation Loss,█▁
batch_size,▁
Epoch,2
Test Loss,0.03929
Training Loss,0.03786
Validation Loss,0.03949
batch_size,5000
